In [ ]:
!pip install transformers datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.0 MB/s eta 0:00:00


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead")

GPU is available


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

drive.mount('/content/drive')
dataset = pd.read_pickle("/content/drive/MyDrive/training.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split

# stratified splitting into training set and test set
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset['label'])
val_dataset = test_dataset

train_dataset['label'].value_counts(),val_dataset['label'].value_counts()

(False    6400
 True      144
 Name: label, dtype: int64,
 False    1600
 True       36
 Name: label, dtype: int64)

In [ ]:
# split the notes from each True patient
train_negatives = train_dataset[train_dataset["label"] != True]
train_positives = train_dataset[train_dataset["label"] == True]
p_texts_train = train_positives['text']
p_labels_train = train_positives['label']

def separate_notes(note):
  notes = note.split("Note")
  notes = [note.strip() for note in notes if note.strip()]
  return notes

for item in p_texts_train:
  notes = separate_notes(item)
  for note in notes:
    new_row = {"text": "Note " + note, "label": True}
    train_negatives = train_negatives.append(new_row, ignore_index=True)

train_dataset = train_negatives

# convert True and False into 1 and 0
label_encoder = LabelEncoder()
train_labels = train_dataset['label']
train_encoded_labels = label_encoder.fit_transform(train_labels)
train_dataset['label'] = train_encoded_labels

train_dataset['label'].value_counts()

Streaming output truncated to the last 5000 lines.
<ipython-input-6-9b8642dec30b>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_negatives = train_negatives.append(new_row, ignore_index=True)
<ipython-input-6-9b8642dec30b>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_negatives = train_negatives.append(new_row, ignore_index=True)
<ipython-input-6-9b8642dec30b>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_negatives = train_negatives.append(new_row, ignore_index=True)
<ipython-input-6-9b8642dec30b>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_negatives = train_negatives.append(new_row, ignore_index

0    6400
1    2879
Name: label, dtype: int64

In [ ]:
val_negatives = val_dataset[val_dataset["label"] != True]
val_positives = val_dataset[val_dataset["label"] == True]

p_texts_val = val_positives['text']
p_labels_val = val_positives['label']

def separate_notes(note):
  notes = note.split("Note")
  notes = [note.strip() for note in notes if note.strip()]
  return notes

for item in p_texts_val:
  notes = separate_notes(item)
  for note in notes:
    new_row = {"text": "Note " + note, "label": True}
    val_negatives = val_negatives.append(new_row, ignore_index=True)

val_dataset = val_negatives

# convert True and False into 1 and 0
label_encoder = LabelEncoder()
val_labels = val_dataset['label']
val_encoded_labels = label_encoder.fit_transform(val_labels)
val_dataset['label'] = val_encoded_labels

val_dataset['label'].value_counts()

<ipython-input-7-557b67d2b01c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val_negatives = val_negatives.append(new_row, ignore_index=True)
<ipython-input-7-557b67d2b01c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val_negatives = val_negatives.append(new_row, ignore_index=True)
<ipython-input-7-557b67d2b01c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val_negatives = val_negatives.append(new_row, ignore_index=True)
<ipython-input-7-557b67d2b01c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val_negatives = val_negatives.append(new_row, ignore_index=True)
<ipython-input-7-557b67d2b01c>:16: FutureWarning: The frame.

0    1600
1     716
Name: label, dtype: int64

In [ ]:
train_notes = train_dataset.text.values
train_labels = train_dataset.label.values
val_notes = val_dataset.text.values
val_labels = val_dataset.label.values


In [ ]:
from transformers import AutoTokenizer, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
train_input_ids = []
train_attention_masks = []

# tokenize the notes
for note in train_notes:
    encoded_dict = tokenizer.encode_plus(
                        note,
                        add_special_tokens = True, #'[CLS]'and'[SEP]'
                        max_length = 512,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        truncation = True,
                        return_tensors = 'pt',
                   )
    train_input_ids.append(encoded_dict['input_ids'])
    train_attention_masks.append(encoded_dict['attention_mask'])

train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

# ----------------------------------------------------------------------------------------------------

# val_input_ids = []
# val_attention_masks = []

# for note in val_notes:
#     encoded_dict = tokenizer.encode_plus(
#                         note,
#                         add_special_tokens = True, #'[CLS]'and'[SEP]'
#                         max_length = 512,
#                         pad_to_max_length = True,
#                         return_attention_mask = True,
#                         truncation = True,
#                         return_tensors = 'pt',
#                    )
#     val_input_ids.append(encoded_dict['input_ids'])
#     val_attention_masks.append(encoded_dict['attention_mask'])

# val_input_ids = torch.cat(val_input_ids, dim=0)
# val_attention_masks = torch.cat(val_attention_masks, dim=0)
# val_labels = torch.tensor(val_labels)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
# val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

# validation_dataloader = DataLoader(
#             val_dataset,
#             sampler = SequentialSampler(val_dataset),
#             batch_size = batch_size
#         )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=2, output_attentions=False, output_hidden_states=False).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F

optimizer = AdamW(model.parameters(),
            lr = 1e-5, # args.learning_rate - default is 5e-5
            eps = 1e-8 # args.adam_epsilon  - default is 1e-8
            )

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
# loss_fn = torch.nn.NLLLoss(weight=torch.tensor([0.7247, 1.6127]).to(device))
loss_fn = torch.nn.NLLLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import random
import numpy as np

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in range(epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
    print('Training...')
    total_train_loss = 0

    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()
        results = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # loss = results['loss']
        logits = results['logits']
        # loss = loss_fn(logits, b_labels)
        # print(logits)
        log_prob = F.log_softmax(logits, dim=-1)
        # print(log_prob)
        loss = loss_fn(log_prob, b_labels)
        loss.backward()
        print(loss)

        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        total_train_loss += loss.item()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))


======== Epoch 1 / 1 ========
Training...
tensor(1.0605, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9258, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9012, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6893, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8049, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6580, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6661, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6173, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6015, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6741, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4811, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6548, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6886, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5800, device='cuda:0', grad_fn=<NllLossBackward0

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0
P = 0
N = 0
total = 0
correct = 0
total_eval_loss = 0

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def stats(logits, labels):
    global TP, FP, TN, FN, P, N, correct, total
    # print(logits)
    print(F.softmax(logits, dim=-1))
    # val_y_pred = np.argmax(logits, axis=1).flatten()
    prob = F.softmax(logits, dim=-1).numpy()
    val_y_pred = np.argmax(prob, axis=1).flatten()
    val_y = labels.flatten()
    # print(val_y_pred, val_y)

    for i in range(len(val_y)):
        if val_y[i] == val_y_pred[i]:
          correct += 1
        if val_y[i] == val_y_pred[i] and val_y[i] == 1:
          TP += 1
        if val_y[i] == val_y_pred[i] and val_y[i] == 0:
          TN += 1
        if val_y[i] != val_y_pred[i] and val_y[i] == 1:
          FN += 1
        if val_y[i] != val_y_pred[i] and val_y[i] == 0:
          FP += 1
        if val_y_pred[i] == 1:
          P += 1
        if val_y_pred[i] == 0:
          N += 1
        total+=1

print("")
print("Running Validation...")

model.eval()

for batch in validation_dataloader:

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        val_results = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # loss = val_results['loss']
        logits = val_results['logits']
        log_prob = F.log_softmax(logits, dim=-1)
        loss = loss_fn(log_prob, b_labels)

    total_eval_loss += loss.item()

    # detach to cpu to calculate stats
    logits = logits.detach().cpu()
    label_ids = b_labels.to('cpu').numpy()

    #
    stats(logits, label_ids)

print(total_eval_loss / len(validation_dataloader))
print(len(validation_dataloader))


Running Validation...
tensor([[0.9297, 0.0703],
        [0.9986, 0.0014],
        [0.9975, 0.0025],
        [0.9983, 0.0017],
        [0.9972, 0.0028],
        [0.9984, 0.0016],
        [0.9986, 0.0014],
        [0.7518, 0.2482],
        [0.9982, 0.0018],
        [0.9978, 0.0022],
        [0.9958, 0.0042],
        [0.9980, 0.0020],
        [0.9924, 0.0076],
        [0.9957, 0.0043],
        [0.9982, 0.0018],
        [0.9986, 0.0014]])
tensor([[0.9982, 0.0018],
        [0.9712, 0.0288],
        [0.9984, 0.0016],
        [0.9985, 0.0015],
        [0.9979, 0.0021],
        [0.9970, 0.0030],
        [0.9966, 0.0034],
        [0.9976, 0.0024],
        [0.9984, 0.0016],
        [0.9958, 0.0042],
        [0.9967, 0.0033],
        [0.9982, 0.0018],
        [0.9973, 0.0027],
        [0.9982, 0.0018],
        [0.9986, 0.0014],
        [0.9780, 0.0220]])
tensor([[0.9983, 0.0017],
        [0.9978, 0.0022],
        [0.9978, 0.0022],
        [0.9973, 0.0027],
        [0.9982, 0.0018],
        [0.99

In [ ]:
TP,FP,TN,FN,P,N,correct,total

(686, 3, 1597, 30, 689, 1627, 2283, 2316)

In [ ]:
test_label_encoder = LabelEncoder()
test_labels = test_dataset['label']
test_encoded_labels = test_label_encoder.fit_transform(test_labels)
test_dataset['label'] = test_encoded_labels

test_dataset['label'].value_counts()

0    1600
1      36
Name: label, dtype: int64

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0
P = 0
N = 0
total = 0
correct = 0

def separate_notes(note):
  notes = note.split("Note")
  notes = [note.strip() for note in notes if note.strip()]
  return notes

def stats(label, pred_label):
    global TP, FP, TN, FN, P, N, correct, total
    if label == pred_label:
      correct += 1
    if label == pred_label and label == 1:
      TP += 1
    if label == pred_label and label == 0:
      TN += 1
    if label != pred_label and label == 1:
      FN += 1
    if label != pred_label and label == 0:
      FP += 1
    if pred_label == 1:
      P += 1
    if pred_label == 0:
      N += 1
    total+=1

threshold = 0.7

for _, row in test_dataset.iterrows():
    notes = separate_notes(row[0])
    trues = 0
    falses = 0

    for note in notes:
      note_input_ids = []
      note_attention_masks = []
      encoded_dict = tokenizer.encode_plus(
                        note,
                        add_special_tokens = True, #'[CLS]'and'[SEP]'
                        max_length = 512,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        truncation = True,
                        return_tensors = 'pt',
                   )
      note_input_ids.append(encoded_dict['input_ids'])
      note_attention_masks.append(encoded_dict['attention_mask'])

      note_input_ids = torch.cat(note_input_ids, dim=0).to(device)
      note_attention_masks = torch.cat(note_attention_masks, dim=0).to(device)

      with torch.no_grad():
        val_results = model(note_input_ids, token_type_ids=None, attention_mask=note_attention_masks)
      logits = val_results['logits']

      logits = logits.detach().cpu()
      prob = F.softmax(logits, dim=-1).numpy()
      # val_y_pred = np.argmax(prob, axis=1).flatten()
      val_y_pred = np.where(prob.max(axis=1) > threshold, prob.argmax(axis=1), 0)
      print(prob, val_y_pred)
      if val_y_pred == 0:
        falses+=1
      else:
        trues+=1

    output = 1 if (trues / (falses + trues)) > 0.95 else 0

    stats(row[1], output)





Streaming output truncated to the last 5000 lines.
[[0.03217887 0.9678211 ]] [1]
[[0.26208815 0.7379119 ]] [1]
[[0.06091363 0.9390864 ]] [1]
[[0.02814801 0.97185206]] [1]
[[0.0317223 0.9682777]] [1]
[[0.5013436  0.49865633]] [0]
[[0.01906482 0.98093516]] [1]
[[0.01819662 0.9818034 ]] [1]
[[0.04262724 0.9573727 ]] [1]
[[0.01639031 0.9836097 ]] [1]
[[0.10350043 0.89649963]] [1]
[[0.20499235 0.7950077 ]] [1]
[[0.03190126 0.9680988 ]] [1]
[[0.03391712 0.96608293]] [1]
[[0.01248774 0.9875123 ]] [1]
[[0.04263016 0.95736986]] [1]
[[0.95126396 0.04873602]] [0]
[[0.8894214  0.11057857]] [0]
[[0.02821275 0.9717872 ]] [1]
[[0.00722757 0.9927724 ]] [1]
[[0.00869689 0.9913031 ]] [1]
[[0.00547108 0.99452895]] [1]
[[0.00961984 0.99038017]] [1]
[[0.7414599 0.2585401]] [0]
[[0.00949519 0.9905048 ]] [1]
[[0.43772152 0.5622785 ]] [0]
[[0.01529901 0.98470104]] [1]
[[0.04928075 0.9507193 ]] [1]
[[0.00882754 0.9911725 ]] [1]
[[0.00500806 0.99499196]] [1]
[[0.01675988 0.98324007]] [1]
[[0.01477105 0.98522896

In [ ]:
TP,FP,TN,FN,P,N,correct,total

(9, 55, 1545, 27, 64, 1572, 1554, 1636)

In [ ]:
F1 = 2/((TP+FP)/TP + P/TP)
acc = correct / total

F1, acc

(0.140625, 0.9498777506112469)

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/finetuned_biobert_final.pt')